- DB서버의 정보
    - 주소
    - port
    - user명
    - 비밀번호
    - 데이터베이스명
- 서버의 정보들은 외부에 노출이 되면 보안적으로 문제 발생할 수 있다. 
    - 서버의 정보와 같이 외부의 노출이 되는 데이터는 dotenv를 이용하여 데이터를 숨겨서 사용 ( 환경 변수 사용 )

In [2]:
# dotenv 라이브러리 설치 
# !pip install python-dotenv

In [3]:
import pandas as pd 
import pymysql 
import os 
from dotenv import load_dotenv

In [4]:
# .env 파일을 환경 변수에 등록한다. 
load_dotenv()

True

In [6]:
# 등록된 환경 변수에 접근 
# 획득하다(get) + 환경변수(env)
# getenv( {변수명} ) -> return data의 type이 str
os.getenv('port')

'3306'

In [7]:
# DB 서버와의 연결 
# mac 사용자는 터미널에서 brew services start mysql
_db = pymysql.connect(
    host = os.getenv('host'), 
    port = int(os.getenv('port')), 
    user = os.getenv('user'), 
    password = os.getenv('pw'), 
    db = os.getenv('db_name')
) 

In [8]:
# 가상 공간 cursor 생성 ( 연결된 데이터베이스로부터 생성 )
cursor = _db.cursor( pymysql.cursors.DictCursor )

In [12]:
create_user_query = """
    CREATE TABLE if not exists `user_info`
    (
        `id` varchar(35) primary key, 
        `password` varchar(45) not null, 
        `name` varchar(45), 
        `birth` varchar(10) 
    )
"""
cursor.execute(create_user_query)

0

In [ ]:
# query문 안에 유저가 입력한 데이터를 대입하여 query 실행 
# 회원 가입을 설정 
# 입력 데이터가 4개 id, password, name, birth
input_id = input('id를 입력하시오')
input_pass = input('password를 입력하시오')
input_name = input('name을 입력하시오')
input_birth = input('birth를 입력하시오')

# 회원 정보를 저장하는 query 생성 
signup_query = """
    insert into 
    `user_info`
    values 
    ( %s, %s, %s, %s )
"""

# signup_query와 데이터를 이용하여 execute() 사용
cursor.execute( signup_query, 
               [input_id, input_pass, input_name, input_birth] )

1

- CUD(insert, update, delete)
    - query 작성
        - 데이터가 존재한다면 데이터들을 리스트나 튜플의 형태로 생성 
    - execute() 함수를 이용하여 cursor에 질의를 보낸다. 
        - cursor에 존재하는 table의 data가 변화
    - DB server와 cursor를 동기화 (.commit())
    - DB server와의 연결을 종료 
- R(select)
    - query 작성 
        - 데이터가 존재한다면 데이터들을 리스트나 튜플의 형태로 생성 
    - execute() 함수를 이용하여 cursor에 질의를 보낸다. 
        - cursor에 존재하는 table의 data의 변화 X
    - execute() 함수의 결과를 불러와서 변수에 저장 (fetchall()) 
    - DB server와의 연결을 종료

In [15]:
# query문이 select문인가?
test_qeury = """
    SELECT 
    * 
    from 
    `table`
"""

In [22]:
# query문이 select 인가?
# 좌우의 공백을 제거(strip()) -> 위치를 이용해서 문자를 나눠준다. -> 
# 문자열을 대문자, 소문자 변환 -> 비교
text_data = 'select'
test_qeury.strip().lower()[ : len(text_data) ] == text_data

True

In [26]:
# 문자를 소문자 변환(lower()) -> 
# 공백을 기준으로 데이터를 나눠준다. ( split() ) ->
# 첫번째 원소[0]를 선택 -> 비교
test_qeury.lower().split()[0] == text_data

True

In [27]:
# 문자를 소문자 변환(lower()) ->
# 좌우의 공백을 제거(strip()) ->
# 시작하는 문자가 text_data와 같은가? 함수를 호출(startswith())
test_qeury.lower().strip().startswith(text_data)

True

In [30]:
# CRUD query를 모두 사용할 수 있는 함수를 생성 
# 매개변수 2개 -> query문, query에 대입이 되는 데이터들(가변)
def sql_qeury(
        _query, *_data_list
):
    # query문을 실행을 시도한다. 
    try:
        # cursor에 질의 보낸다. 
        cursor.execute(_query, _data_list)
    # execute()함수가 에러가 발생한다면
    except Exception as e:
        print(e)
        # 함수를 강제로 종료하면서 메시지 되돌려준다. 
        return "Query Error"
    # _query는 sql query문이 문자로 입력이되는 부분 
    # _query가 select문인가?
    if _query.lower().strip().startswith('select'):
        # select문에 대한 결과를 불러온다. 
        result = cursor.fetchall()
    else:
        result = 'Query OK'
    return result

In [32]:
test_query = """
    select 
    * 
    from 
    `user_info`
"""
sql_qeury(test_query)

[{'id': 'test', 'password': '1234', 'name': 'kim', 'birth': '900101'},
 {'id': 'test1234', 'password': '0000', 'name': 'kim', 'birth': '900101'},
 {'id': 'test2', 'password': '1234', 'name': 'park', 'birth': '000101'}]

In [36]:
# 로그인 과정 
# 유저가 입력한 id, password를 이용해서 데이터베이스에 데이터가 존재하는가?
input_id = input("id 입력하시오")
input_pass = input('password 입력하시오')

login_query = """
    select * from 
    `user_info`
    where 
    `id` = %s 
    and
    `password` = %s
"""

login_result = sql_qeury(login_query, input_id, input_pass)

In [37]:
if login_result:
    # login_result에 데이터가 존재한다면 -> 길이가 1이 아닌
    print('로그인 성공')
else:
    # login_result에 데이터가 존재하지 않는다면 -> 길이가 0인
    print('로그인 실패')

로그인 실패


In [38]:
input_id = 'test0000'
input_pass = '1234'
input_name = 'park'
input_birth = '000101'

insert_query = """
    insert into 
    `user_info`
    values 
    (%s, %s, %s, %s)
"""

sql_qeury(insert_query, input_id, input_pass, input_name, input_birth)

'Query OK'

In [39]:
# 데이터베이스 서버와 cursor를 동기화 하고 서버와의 연결을 종료하는 함수 생성 
def commit_db():
    # cursor와 데이터베이스 서버와의 동기화
    _db.commit()
    print('Commit 완료')
    # 데이터베이스 서버와의 연결을 종료 
    _db.close()
    print('서버와의 연결 종료')

In [40]:
commit_db()

Commit 완료
서버와의 연결 종료


In [97]:
# 데이터베이스 서버와 연결하여 query문을 사용하고 서버와의 연결을 종료하는 class 생성

# class 선언 
class MyDB:
    # 생성자 함수 
    # class 내부에서 사용하려는 변수의 데이터를 대입하는 함수 
    # class 생성 될때 한번만 실행이 되는 함수 
    # 입력 받을 데이터는 데이터베이스 서버의 정보 -> 기본값 설정(로컬 피씨의 데이터베이스 정보)
    def __init__(
            self, 
            _host = '127.0.0.1', 
            _port = 3306, 
            _user = 'root', 
            _pw = '1234', 
            _db_name = 'multicam'
    ):
        # self.변수를 생성 
        # class에서 사용할 서버의 정보를 변수에 저장 
        self.host = _host
        self.port = _port
        self.user = _user
        self.pw = _pw
        self.db_name = _db_name
    
    # query문과 데이터를 입력받아서 데이터베이스 서버에 질의를 보내는 함수 
    def sql_query(
            self, 
            _query, 
            *_data_list
    ):
        # _query 매개변수는 기본값이 존재하지 않으므로 필수 입력 공간 
        # _data_list는 인자의 개수를 가변으로 받는다. 개수가 0개면 ()을 생성

        # 데이터베이스 서버와의 연결 ( _db 변수를 생성하여 연결 )
        # pymysql.connet() 함수는 입력값이 서버의 정보 -> 
        # 서버의 정보는 self.host, self.port, self.user, self.pw, self.db_name 저장되어 있다. 
        # connect() 함수에 해당 데이터를 입력
        try:
            # self._db라는 변수를 확인
            self._db
            # 해당 변수가 선언이 되어있지 않다면 NameError 발생
        except:
            # self._db라는 변수가 존재하지 않을때 
            self._db = pymysql.connect(
                host = self.host, 
                port = self.port, 
                user = self.user, 
                password = self.pw, 
                db = self.db_name
            )
        # self._db를 이용하여 cursor를 생성 
        # ( class 내부에 다른 함수에서 사용이 가능하도록 self.변수로 등록 )
        self.cursor = self._db.cursor( pymysql.cursors.DictCursor )

        # _query와 _data_list를 이용하여 self.cursor에 질의 보낸다. 
        # 질의를 보내는 과정에서 문제가 발생하면 예외 처리를 한다. 
        try : 
            self.cursor.execute( _query, _data_list )
        except Exception as e:
            print(e)
            # query문에서 문제가 발생했으니 에러 메시지를 돌려주고 함수를 종료 
            return 'Query Error'
        # _query가 만약에 select 문이라면?
        if _query.lower().strip().startswith('select'):
            # query문이 select문인 경우
            # self.cursor에서 결과물을 돌려받는다. 
            result = self.cursor.fetchall()
        else:
            # select문이 아닌 경우 
            result = "Query OK"
        return result
    def db_commit(self):
        try:
            # self._db와 self.cursor를 동기화
            self._db.commit()
            print('commit 완료')
            # 데이터베이스 서버와의 연결을 종료 
            self._db.close()
            print('서버와의 연결 종료')
            # 변수를 삭제 
            del self._db
        except Exception as e:
            print(e)
            # self._db와 self.cursor가 생성되지 않은 상황 
            print("""데이터베이스 서버와의 연결이 없습니다. 
                  sql_query() 함수를 이용하여 서버와의 연결을 해주세요""")

In [98]:
# MyDB class 생성하는데 생성자 함수의 매개변수는 기본값으로 사용
# 로컬 피씨에 있는 데이터베이스에 연결 
db1 = MyDB()

In [99]:
# 외부의 데이터베이스 연결 
db2 = MyDB(
    '192.168.50.40', 
    3306, 
    'test', 
    '1234', 
    'multicam'
)

In [100]:
select_query = """
    select 
    * 
    from 
    `user_info`
"""

In [101]:
db1.sql_query(select_query)

[{'id': 'test', 'password': '1234', 'name': 'kim', 'birth': '900101'},
 {'id': 'test0000', 'password': '1234', 'name': 'park', 'birth': '000101'},
 {'id': 'test1234', 'password': '0000', 'name': 'kim', 'birth': '900101'},
 {'id': 'test2', 'password': '0123', 'name': 'park', 'birth': '000101'}]

In [102]:
db2.sql_query(select_query)

[{'id': 'test', 'password': '1234', 'name': 'kim', 'birth': '900101'},
 {'id': 'test0000', 'password': '1234', 'name': 'park', 'birth': '000101'},
 {'id': 'test1234', 'password': '0000', 'name': 'kim', 'birth': '900101'},
 {'id': 'test2', 'password': '0123', 'name': 'park', 'birth': '000101'}]

In [103]:
update_query = """
    update 
    `user_info`
    set 
    `password` = %s
    where 
    `id` = %s
"""

In [104]:
db1.sql_query(update_query, "0123", "test2")

'Query OK'

In [107]:
db1.sql_query(select_query)

[{'id': 'test', 'password': '1234', 'name': 'kim', 'birth': '900101'},
 {'id': 'test0000', 'password': '1234', 'name': 'park', 'birth': '000101'},
 {'id': 'test1234', 'password': '0000', 'name': 'kim', 'birth': '900101'},
 {'id': 'test2', 'password': '0123', 'name': 'park', 'birth': '000101'}]

In [106]:
db1.db_commit()

commit 완료
서버와의 연결 종료


In [108]:
import database

In [109]:
# database 모듈 안에 있는 MyDB 생성
mod_db = database.MyDB()

In [111]:
# mod_db에 입력이 되는 데이터베이스 서버의 정보는? -> 로컬
select_query2 = """
    select * from `user_info`
"""
db_data = mod_db.sql_query(select_query2)

In [112]:
pd.DataFrame(db_data)

,id,password,name,birth
0,test,1234,kim,900101
1,test0000,1234,park,000101
2,test1234,0000,kim,900101
3,test2,0123,park,000101


In [113]:
# database.py 파일을 python 환경변수에 이동

# 환경 변수의 목록을 확인하기 위해 라이브러리 하나 호출 
import sys

In [116]:
sys.path[5].replace('\\', '/')

'C:/Users/ekfla/AppData/Roaming/Python/Python311/site-packages'